## This is going to be my first neuralnetwork, i dont know how its going to perform over the given data set , but since i heard that . Neural networks are good at capturing non-linearity. I will use all the trciks from the andrew Ng course i am going to give it just 2 hours and i hope that you my creation will outperform each and every of my creation till now.

### FIrstly we will try without any feature transformation , then we will try out with feture transformation, alright.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from keras.models import Sequential
from keras.layers import Dense

# fix random seed for reproducibility
np.random.seed(7)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['train.csv', 'test.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train['age'].mode()
len(train[train['age'] == 26])/ len(train)

0.035490177314175565

In [4]:
train['age'].fillna(26, inplace = True)
test['age'].fillna(26, inplace = True)
train['trainee_engagement_rating'].fillna(1, inplace = True)
test['trainee_engagement_rating'].fillna(1, inplace = True)

In [5]:
col_drop = ['program_id', 'id', 'test_id','trainee_id']    ## we will eliminate age from here.since wew ill use ut as it is.
train_o = train.drop(col_drop , axis = 1)
print(train_o.shape)
train_o.head()

(73147, 12)


,program_type,program_duration,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,Y,136,offline,intermediate,M,Matriculation,3,26.0,5,N,1.0,0
1,T,131,offline,easy,F,High School Diploma,4,30.0,2,N,3.0,1
2,Z,120,online,easy,M,Matriculation,1,45.0,1,N,2.0,1
3,T,117,offline,easy,F,Matriculation,3,26.0,4,N,1.0,1
4,V,131,offline,intermediate,F,High School Diploma,1,43.0,2,N,4.0,1


In [6]:
## for the actual test data
ids = test['id']
col_drop = ['program_id', 'id', 'test_id', 'trainee_id']
tests = test.drop(col_drop , axis = 1)
print(tests.shape)
tests.head()

(31349, 11)


,program_type,program_duration,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,T,131,offline,intermediate,F,Matriculation,3,50.0,2,N,4.0
1,Y,135,online,easy,M,Bachelors,3,22.0,4,N,4.0
2,Y,120,online,easy,M,Matriculation,3,24.0,2,N,3.0
3,V,122,offline,vary hard,F,High School Diploma,1,27.0,2,N,2.0
4,V,122,offline,intermediate,F,Matriculation,3,18.0,2,N,2.0


In [7]:
from sklearn.preprocessing import LabelEncoder
## Lets tyy out some feture transformation ideas.
## replace masters with bachelors
## if total_programs_enrolled >5 then assign 5 , jsut to keep the distribution uniform idk if it will work or not.
def feat_transform(data):
    ## we will try this feature transformation in the next try
    data['total_programs_enrolled'] = [5 if i > 5 else i for i in data.total_programs_enrolled]
    ## made the dictionay for the difficulty_level, education

    ## use labelEncoder for test_type, gender, is_handicapped
    label = LabelEncoder()
    data['test_type'] = label.fit_transform(data['test_type'])
    data['gender'] = label.fit_transform(data['gender'])
    data['is_handicapped'] = label.fit_transform(data['is_handicapped'])

In [8]:
feat_transform(train_o)

In [9]:
feat_transform(tests)
print(set(tests['education']))
print(set(tests['total_programs_enrolled']))

{'Matriculation', 'Bachelors', 'High School Diploma', 'No Qualification', 'Masters'}
{1, 2, 3, 4, 5}


In [10]:
dict_level =  {'easy':0, 'intermediate':1, 'hard':2, 'vary hard':3}
train_o['difficulty_level'] = train['difficulty_level'].map(dict_level)
dict_edu = {'No Qualification':0, 'High School Diploma':1, 'Matriculation':2, 'Bachelors':3, 'Masters':4}
train_o['education'] = train['education'].map(dict_edu)
train_o.head()

,program_type,program_duration,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,Y,136,0,1,1,2,3,26.0,5,0,1.0,0
1,T,131,0,0,0,1,4,30.0,2,0,3.0,1
2,Z,120,1,0,1,2,1,45.0,1,0,2.0,1
3,T,117,0,0,0,2,3,26.0,4,0,1.0,1
4,V,131,0,1,0,1,1,43.0,2,0,4.0,1


In [11]:
dict_level =  {'easy':0, 'intermediate':1, 'hard':2, 'vary hard':3}
tests['difficulty_level'] = test['difficulty_level'].map(dict_level)
dict_edu = {'No Qualification':0, 'High School Diploma':1, 'Matriculation':2, 'Bachelors':3, 'Masters':4}
tests['education'] = test['education'].map(dict_edu)
tests.head()

,program_type,program_duration,test_type,difficulty_level,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,T,131,0,1,0,2,3,50.0,2,0,4.0
1,Y,135,1,0,1,3,3,22.0,4,0,4.0
2,Y,120,1,0,1,2,3,24.0,2,0,3.0
3,V,122,0,3,0,1,1,27.0,2,0,2.0
4,V,122,0,1,0,2,3,18.0,2,0,2.0


In [12]:
columns = ['program_type']
train_orig = pd.get_dummies(train_o , columns = columns)    ## remove city_tier give it a try 
print(train_orig.shape) 

(73147, 18)


In [13]:
columns = ['program_type']
test_orig = pd.get_dummies(tests, columns = columns)      ## remove n city tier
print(test_orig.shape)                                    ## add city_tier
print(test_orig.columns)

(31349, 17)
Index(['program_duration', 'test_type', 'difficulty_level', 'gender',
       'education', 'city_tier', 'age', 'total_programs_enrolled',
       'is_handicapped', 'trainee_engagement_rating', 'program_type_S',
       'program_type_T', 'program_type_U', 'program_type_V', 'program_type_X',
       'program_type_Y', 'program_type_Z'],
      dtype='object')


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_curve, classification_report, average_precision_score,roc_auc_score
import matplotlib.pyplot as plt

In [15]:
train_df, test_df = train_test_split(train_orig, test_size = 0.25, random_state= 1984)

In [16]:
y_train = train_df.is_pass
y_val = test_df.is_pass

x_train = train_df.drop(['is_pass'], axis = 1)
x_val = test_df.drop(['is_pass'], axis = 1)

## Make your data ready to feed in ot the neural network

#y_train = np.array(y_train)
#y_val = np.array(y_val)

#x_train = np.array(x_train)
#x_val = np.array(x_val)


In [17]:
print(x_train.shape)

(54860, 17)


In [18]:
from keras import optimizers

In [19]:
## Baseline AUC_score = 0.589
##improvred roc .592 
## loss decreases to .51 but roc .590
## lets try with improved ffeatures

## Create model, add 10 ylayer, usae adam optmizer , we are not adding anything to tackle overfitting (regularization)
## Create model 
#model = Sequential()
## lets make it a ten layer NN
model.add(Dense(48, input_dim = x_train.shape[1], init = 'normal', activation = 'relu'))  ## init = he_normal or glorot_normal
model.add(Dense(36, activation = 'relu'))
model.add(Dense(36, activation = 'relu'))
model.add(Dense(36, activation = 'relu'))
model.add(Dense(36, activation = 'relu'))
model.add(Dense(26, activation = 'relu'))
model.add(Dense(26, activation = 'relu'))
model.add(Dense(26, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer = adam)

model.fit(x_train, y_train, epochs = 600, batch_size = 128)
## loss at the last epoch 0.5517, fom here on we will try to optimize (Simple NN , batch_siz = 32)
## loss at the last epoch 0.5602, fom here on we will try to optimize (Deep NN with optimizer, batch_size = 256)
## loss at the last epoch .5560, fom here on we will try to optimize (Deep NN with optimizer, batch_size = 256, double the iterations )
## loss at the last epoch , fom here on we will try to optimize (Deep NN with optimizer, batch_size = 128, double the iterations )

NameError: name 'model' is not defined

In [20]:
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')

ModuleNotFoundError: No module named 'keras.utils.visualize_util'

In [21]:
predict = model.predict(x_val) #y_val
pre.shape

NameError: name 'model' is not defined

In [22]:
pre = predict[:,0]
## label it as 0 and 1 
pre01 = np.where(pre > .5, 1, 0)

NameError: name 'predict' is not defined

In [23]:
from sklearn.metrics import accuracy_score
accuracy01 = accuracy_score(y_val, pre01)
print('accuracy of my neural network', accuracy01)

NameError: name 'pre01' is not defined

In [24]:
roc_score = roc_auc_score(y_val, pre01)
print('ROC score', roc_score)   ## it can even go bad for the actual test set

NameError: name 'pre01' is not defined

In [25]:
## Also build the confusion matrix 
import seaborn as sns
cm = confusion_matrix(y_val, pre01)
plt.figure(figsize = (10,10))
sns.heatmap(cm, annot = True, cmap = 'Blues', fmt = 'd', vmin = .2)
plt.title('Confusion_Matrix')


NameError: name 'pre01' is not defined

## As the confusion mat depicts even if our accuracy is good but still we are not i a  good place 
## FN :4099, FP : 1000  simple NN
## FN: 4243 FP: 939 Deep NN with optimizer
## FN: 4073 FP: 1188 Deep NN with optimizer .5844

In [66]:
## Build a LGBM
## Reference https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.00003
params['boosting_type'] = 'dart'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['max_bin'] = 1000
params['sub_feature'] = 0.5
params['num_leaves'] = 500
params['min_data'] = 2000
params['max_depth'] = 12
clf = lgb.train(params, d_train, 1000)

In [67]:
y_pred=clf.predict(x_val)
#convert into binary values
for i in range(0,x_val.shape[0]):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [68]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
cm

array([[   74,  5446],
       [   18, 12749]])

In [62]:

w = roc_auc_score(y_val, y_pred)
w

0.52754217735261766